<a href="https://colab.research.google.com/github/BenxiaHu/DeepLearning/blob/main/pytorch_2D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network


Dependencies:
* torch: 0.1.11
* matplotlib

# step0: import python packages

In [44]:
import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.nn.functional as Fun
from numpy import array
import pandas as pd
import numpy as np
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import requests
torch.manual_seed(1)


# step1: import data

In [45]:
seqid="https://raw.githubusercontent.com/BenxiaHu/DeepLearning/main/sequences.txt"
labelid="https://raw.githubusercontent.com/BenxiaHu/DeepLearning/main/labels.txt"
input = pd.read_csv(seqid,header=None)
label = pd.read_table(labelid,header=None)

print(input)

DNA = np.zeros(shape=(len(input),len(input[0][0]), 4))
labelid = np.zeros(shape=(len(input),))
print(DNA.shape)
print(labelid.shape)

for i in range(input.shape[0]):
    i=0
    seq_array = array(list(input[0][i]))
    #integer encode the sequence
    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)
    #one hot the sequence
    onehot_encoder = OneHotEncoder(sparse_output=False)
    #reshape because that's what OneHotEncoder likes
    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)
    DNA[i] = onehot_encoded_seq
    labelid[i] = label[0][i]

DNA = torch.tensor(DNA)
labelid =  torch.tensor(labelid)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    DNA = DNA.to("cuda:0")
    labelid =  labelid.to("cuda:0")
print(DNA.is_cuda)

print(np.shape(DNA))

torch.Size([2000, 50, 4])


# step2: split the data into training, test and prediction data

In [74]:
input_tensor = DNA
label_tensor = labelid
# pick 1000 samples as training data
print(input_tensor[0:1000].shape)
print(label_tensor[0:1000].shape)
torch_dataset = Data.TensorDataset(input_tensor[0:1000], label_tensor[0:1000])

# Hyper Parameters
EPOCH = 100              # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 100
LR = 0.01                # learning rate
# Data Loader for easy mini-batch return in training, the image batch shape will be (100, 1, 50, 4)
train_loader = Data.DataLoader(dataset=torch_dataset, batch_size=BATCH_SIZE, shuffle=True)
#for i, j in enumerate(train_loader):
    #x, y = j
    #print('batch:{0} x:{1}  y: {2}'.format(i, x, y))
    #print(x.shape)
    #print(y.shape)

# pick 600 samples as testing data
# shape from (600, 50, 4) to (600, 1, 50, 4), value in range(0,1)
test_x = Variable(torch.unsqueeze(input_tensor[1000:1600], dim=1)).type(torch.FloatTensor)[:1400]
test_y = label_tensor[1000:1600]

print(test_x.shape)
print(test_y.shape)

# pick 400 samples as prediction data
test_x2 = Variable(torch.unsqueeze(input_tensor, dim=1)).type(torch.FloatTensor)[1600:2000]
test_y2 = label_tensor[1600:2000]



# step4: build the 1D-CNN model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (50, 1, 50, 4)
            nn.Conv2d(
                in_channels=1,              # input height
                out_channels=16,            # n_filters
                kernel_size=3,              # filter size
                stride=1,                   # filter movement/step
                padding=(1,2)                   # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (16, 50, 4)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(
                2,
                stride=2,
                padding=1)    # choose max value in 2x2 area, output shape (16, 25, 2)
        )
        self.conv2 = nn.Sequential(         # input shape (16, 25, 2)
            nn.Conv2d(
                in_channels=16,              # input height
                out_channels=8,            # n_filters
                kernel_size=3,              # filter size
                stride=1,                   # filter movement/step
                padding=(1,2)),                 # output shape (32, 27, 4)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(2,2),                # output shape (32, 13, 3)
        )
        self.out = nn.Linear(8 * 13 * 3, 2)   # fully connected layer, output 2 classes
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, 32, 12, 1))
        output = self.out(x)
        return output, x    # return x for visualization
